In [ ]:
%pip install owslib shapely matplotlib

### Imports

In [ ]:
from owslib.wps import WebProcessingService, ComplexDataInput, monitorExecution
from owslib import wps
import matplotlib
from matplotlib import pyplot
from shapely.geometry import shape
import json
import pandas as pd
import rasterio
import logging
import sys
import io
from ipyleaflet import Map, GeoJSON, basemaps

In [ ]:
# Set up owslib logging, to set relevant output
logger = logging.getLogger("owslib")
logger.setLevel(logging.WARNING)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARNING)
logger.addHandler(handler)

### Consts - Change these to affect the data retrieved

In [ ]:
ENDPOINT="https://funcwps.ternlandscapes.org.au/wps/"
DATASET="aet"  # options are "smips" or "aet"
LAYER="ETa"  # options are "totalbucket" or "SMindex" for smips, or "ETa" for "aet".
ATDATE="2019-01-01"
# Enter your POLYGON as GeoJSON format.
MY_POLYGON='''{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              152.4076151068369,
              -27.740499600282057
            ],
            [
              153.4719537627558,
              -27.740499600282057
            ],
            [
              153.4719537627558,
              -27.093849063222414
            ],
            [
              152.4076151068369,
              -27.093849063222414
            ],
            [
              152.4076151068369,
              -27.740499600282057
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              153.04560361385302,
              -30.261677316183146
            ],
            [
              153.01223568656428,
              -30.274209977967658
            ],
            [
              152.97925359656017,
              -30.306757809371064
            ],
            [
              153.00516809584911,
              -30.388080106869154
            ],
            [
              153.11038986207154,
              -30.357268040135885
            ],
            [
              153.14690574743526,
              -30.313551938010704
            ],
            [
              153.12216917993214,
              -30.248449987166744
            ],
            [
              153.04560361385302,
              -30.261677316183146
            ]
          ]
        ],
        "type": "Polygon"
      }
    },
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              150.9019211334824,
              -34.00680553492937
            ],
            [
              151.0874349541068,
              -34.00971216084313
            ],
            [
              151.31917860367787,
              -33.89045896204837
            ],
            [
              151.25957039365034,
              -33.692305214021
            ],
            [
              150.78621107872112,
              -33.642695168765655
            ],
            [
              150.71257740751076,
              -33.92537963226312
            ],
            [
              150.9019211334824,
              -34.00680553492937
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}'''

### Define a connection to the server

In [ ]:
server = WebProcessingService(ENDPOINT, verbose=False)

In [ ]:
print('WPS Identification type: %s' % server.identification.type)
print('WPS Identification title: %s' % server.identification.title)
print('WPS Identification abstract: %s' % server.identification.abstract)
for operation in server.operations:
    print('WPS Operation: %s' % operation.name)
for process in server.processes:
    print('WPS Process: identifier=%s title=%s' % (process.identifier, process.title))

WPS Identification type: WPS
WPS Identification title: TERN Landscapes Functional Data WPS Server
WPS Identification abstract: The Functional Data Processing Server from TERN Landscapes
WPS Operation: GetCapabilities
WPS Operation: DescribeProcess
WPS Operation: Execute
WPS Process: identifier=sleep title=Sleep Process
WPS Process: identifier=temporalDrill title=Temporal
WPS Process: identifier=subset1d title=SubSet 1D
WPS Process: identifier=subset2d title=SubSet 2D
WPS Process: identifier=stats2d title=Stats 2D


In [ ]:
process = server.describeprocess("stats2d")
print('WPS Process: identifier=%s' % process.identifier)
print('WPS Process: title=%s' % process.title)
print('WPS Process: abstract=%s' % process.abstract)
for input in process.dataInputs:
    print('Process input: identifier=%s, data type=%s, minOccurs=%d, maxOccurs=%d' % (input.identifier, input.dataType, input.minOccurs, input.maxOccurs))
for output in process.processOutputs:
    print('Process output: identifier=%s, data type=%s' % (output.identifier, output.dataType))

WPS Process: identifier=stats2d
WPS Process: title=Stats 2D
WPS Process: abstract=Return CSV file containing stats within given 2D polygon.
Process input: identifier=datasetId, data type=string, minOccurs=1, maxOccurs=1
Process input: identifier=atDate, data type=date, minOccurs=1, maxOccurs=1
Process input: identifier=polygon, data type=ComplexData, minOccurs=1, maxOccurs=1
Process output: identifier=download_link, data type=string
Process output: identifier=csv, data type=ComplexData


### Optional - Visualize shape

In [ ]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=[-25.128, 135.588], zoom=4)
m.add_layer(GeoJSON(data=json.loads(MY_POLYGON)))
m

Map(center=[-25.128, 135.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

### Set up to send request, first create inputs

In [ ]:
poly_input = ComplexDataInput(MY_POLYGON, mimeType="application/vnd.geo+json", schema="http://geojson.org/geojson-spec.html#polygon")

### Trigger process to begin (process executes asynchronously)

In [ ]:
resp = server.execute(
    "stats2d",
    mode="async",
    inputs=[("datasetId", f"{DATASET}:{LAYER}"), ("polygon", poly_input), ("atDate", ATDATE)],
    output=[("csv", False, "text/csv")]  # Get stats as embedded file in result
)

### Wait for process to complete (polls every 4 seconds)

In [ ]:
while not resp.isComplete():
    print("Waiting for process to complete...")
    resp.checkStatus(sleepSecs=4)
    print(f"Status: {resp.status}")
print("Done")

Waiting for process to complete...
Status: ProcessStarted
Waiting for process to complete...
Status: ProcessSucceeded
Done


### Extract CSV data and render it

In [ ]:
first_output = next(iter(resp.processOutputs))
data = "".join(d for d in first_output.data)
df = pd.read_csv(io.StringIO(data))
df

,date,feature,min,max,mean,valid,stddev
0,2019-01-01T00:00:00+00:00,1,26.0,7377.0,4056.195808,63.825359,945.964291
1,2019-01-01T00:00:00+00:00,2,0.0,9499.0,3129.108286,73.019760,1063.869223
2,2019-01-01T00:00:00+00:00,0,0.0,9608.0,3290.329767,78.557025,1243.135557
